# Code to Clean The Raw Crime Dataset

## Import Packages and Files

In [9]:
#import necessary packages
import pandas as pd
import calendar

In [11]:
#import raw crime dataset and display. Change File location to location on system
file_location = 'C:/Users/pkhu624/OneDrive - The University of Auckland/399/Crime Data/Raw Crime Data.csv'

data = pd.read_csv(file_location)

display(data)

,Hour,Occurrence Day Of Week,ANZSOC Division,Map Detail Name,ANZSOC Group,Area Unit,Occurrence Hour Of Day,Region,Territorial Authority,Year Month,Mb2014,REGC2014 label,Ta2014 Nam,Number of Victimisations,Victimisations,Latitude,Longitude
0,11,Wed,Acts Intended to Cause Injury,781200.0,Serious Assault Resulting in Injury,Manukau Central.,11,Auckland Region,Auckland.,1/12/2022,781200.0,Auckland Region,Auckland,1,1,-37.012507,174.846544
1,10,Tue,Acts Intended to Cause Injury,781200.0,Common Assault,Manukau Central.,10,Auckland Region,Auckland.,1/01/2023,781200.0,Auckland Region,Auckland,1,1,-37.012507,174.846544
2,9,Fri,Acts Intended to Cause Injury,781200.0,Common Assault,Manukau Central.,9,Auckland Region,Auckland.,1/01/2023,781200.0,Auckland Region,Auckland,1,1,-37.012507,174.846544
3,10,Tue,Sexual Assault and Related Offences,781200.0,Aggravated Sexual Assault,Manukau Central.,10,Auckland Region,Auckland.,1/01/2023,781200.0,Auckland Region,Auckland,1,1,-37.012507,174.846544
4,11,Tue,Sexual Assault and Related Offences,781200.0,Aggravated Sexual Assault,Manukau Central.,11,Auckland Region,Auckland.,1/01/2023,781200.0,Auckland Region,Auckland,1,1,-37.012507,174.846544
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11247,15,Sun,"Abduction, Harassment and Other Related Offenc...",769018.0,Abduction and Kidnapping,Wattle Farm.,15,Auckland Region,Auckland.,1/01/2023,769018.0,Auckland Region,Auckland,1,1,-37.049264,174.893222
11248,12,Sun,"Abduction, Harassment and Other Related Offenc...",184903.0,Abduction and Kidnapping,Glendhu.,12,Auckland Region,Auckland.,1/01/2023,184903.0,Auckland Region,Auckland,2,2,-36.772832,174.709614
11249,14,Wed,Theft and Related Offences,179134.0,"Theft (Except Motor Vehicles), N.E.C.",Northcross.,14,Auckland Region,Auckland.,1/12/2022,179134.0,Auckland Region,Auckland,1,1,-36.721166,174.723429
11250,21,Wed,Acts Intended to Cause Injury,227304.0,Serious Assault Not Resulting in Injury,Waimumu North.,21,Auckland Region,Auckland.,1/01/2023,227304.0,Auckland Region,Auckland,2,2,-36.842189,174.612706


## Column reformatting 

In [12]:
#Change column ANZSOC Division to incidents and each instance reflect what it would be called on the app
data = data.rename(columns={'ANZSOC Division': 'Incidents', 'Year Month': 'Date'})
data['Incidents'] = data['Incidents'].replace({'Abduction, Harassment and Other Related Offences Against a Person': 'Harrassment', 'Acts Intended to Cause Injury': 'Battery', 'Robbery, Extortion and Related Offences': 'Robbery', 'Sexual Assault and Related Offences': 'Sexual Assault', 'Theft and Related Offences': 'Theft', 'Unlawful Entry With Intent/Burglary, Break and Enter': 'Burglary'})

#Delete any unnecessary columns
cols_to_drop = ['ANZSOC Group', 'Map Detail Name', 'Occurrence Hour Of Day', 'Territorial Authority', 'Region', 'REGC2014 label', 'Ta2014 Nam', 'Mb2014', 'Victimisations'] 

data.drop(cols_to_drop, axis=1, inplace=True)

#remove dot at the end of Area unit
data['Area Unit'] = data['Area Unit'].str.strip('.')

#sort by the dataset by Date and then by the Day of the week. 
sorted_df = data.sort_values(['Date', 'Occurrence Day Of Week'])


## Date Reformatting 

In [13]:
#Function to map the correct day for the date 
def change_day_column(dataframe, occurrences):
    for index, row in dataframe.iterrows():
        year = row['Year']
        month = row['Month']
        day_of_week = row['Occurrence Day Of Week']
        # Get the calendar for the given month and year
        _, num_days = calendar.monthrange(year, month)

        # Convert the day of the week to its corresponding index (Monday: 0, Sunday: 6)
        target_day_index = ['mon', 'tue', 'wed', 'thu', 'fri', 'sat', 'sun'].index(day_of_week.lower()[:3])

        days = []
        found_occurrences = 0
        for day in range(1, num_days + 1):
            current_day_index = calendar.weekday(year, month, day)
            if current_day_index == target_day_index:
                found_occurrences += 1
                if found_occurrences in occurrences:
                    days.append(day)
        
        if days:
            day_index = (index + 1) % len(days) - 1
            dataframe.at[index, 'Day'] = days[day_index]
        

In [14]:
#Change the type of Date column to a date type in yyyy/mm/dd format
sorted_df['Date'] = pd.to_datetime(sorted_df['Date'], format='%d/%m/%Y')

#create empty Day column and helper columns to be used in function
sorted_df['Day']= ''
sorted_df['Month'] = sorted_df['Date'].dt.month
sorted_df['Year'] = sorted_df['Date'].dt.year
#specify max number of weeks in a month is 5
number_dates = [1, 2, 3, 4, 5]

#run function to fill empty in empty Day 
change_day_column(sorted_df, number_dates)

#Replacing Dates with the updated dates and dropping the Y,M,D helper columns
sorted_df['Date'] = pd.to_datetime(sorted_df[['Year', 'Month', 'Day']])
sorted_df.drop(['Year', 'Month', 'Day'], axis=1, inplace=True)

#sort by date and then occurrence Day of Week 
sorted_df = sorted_df.sort_values(by=['Date'], ascending = False)

## Removing Duplciate Rows

In [15]:
#Remove Duplciate rows
duplicates_mask = sorted_df.duplicated()
duplicates = sorted_df[duplicates_mask]
print("These are the {} duplicate rows in the data".format(len(duplicates)))
sorted_df = sorted_df.drop_duplicates()
duplicates_mask = sorted_df.duplicated()
duplicates = sorted_df[duplicates_mask]
print("These are now {} duplicate rows in the data".format(len(duplicates)))

These are the 44 duplicate rows in the data
These are now 0 duplicate rows in the data


## Saving Clean Data

In [17]:
# Save the DataFrame to a CSV file
sorted_df.to_csv('Clean Crime Data.csv', index=False)
display(sorted_df)

,Hour,Occurrence Day Of Week,Incidents,Area Unit,Date,Number of Victimisations,Latitude,Longitude
3659,16,Tue,Theft,Royal Oak,2023-01-31,1,-36.909989,174.774556
9834,1,Tue,Burglary,Mt Wellington South,2023-01-31,1,-36.917973,174.844986
2419,6,Tue,Theft,Dingwall,2023-01-31,1,-36.966078,174.863773
6269,14,Tue,Theft,Blockhouse Bay,2023-01-31,1,-36.925920,174.704431
11034,6,Tue,Burglary,Parnell West,2023-01-31,1,-36.852189,174.778878
...,...,...,...,...,...,...,...,...
3100,9,Thu,Theft,Avondale West,2022-12-01,1,-36.896609,174.687890
9270,2,Thu,Theft,Avondale South,2022-12-01,1,-36.911586,174.697955
3015,11,Thu,Battery,Windy Ridge,2022-12-01,1,-36.796685,174.720010
3000,11,Thu,Theft,Auckland Central East,2022-12-01,1,-36.850308,174.774110
